In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20, 6)

In [2]:
import sys
import os
import json
import torch
import yaml
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from nerfstudio.engine.trainer import Trainer
from nerfstudio.utils.eval_utils import eval_setup

In [3]:
os.chdir('/workspace')
MODEL_PATH = "/workspace/outputs/5_spins_with_noise/plane-nerf/2023-11-27_101812"
INPUT_IMG = "/workspace/plane-nerf/data/5_spins/images/0.png"
TARGET_IMG = "/workspace/plane-nerf/data/5_spins/images/5.png"

In [4]:
config_path = os.path.join(MODEL_PATH, "config.yml")

config = yaml.load(Path(config_path).read_text(), Loader=yaml.Loader)

_, pipeline, _, _ = eval_setup(
                        Path(config_path),
                        test_mode="inference",
                    )

TrainerConfig:
    _target: <class 'nerfstudio.engine.trainer.Trainer'>
    data: /stored_data/5_spins_with_noise
    experiment_name: 5_spins_with_noise
    gradient_accumulation_steps: 1
    load_checkpoint: None
    load_config: None
    load_dir: utputs/5_spins_with_noise/plane-nerf/2023-11-27_101812/nerfstudio_models
    load_scheduler: True
    load_step: None
    log_gradients: False
    logging: LoggingConfig:
        local_writer: LocalWriterConfig:
            _target: <class 'nerfstudio.utils.writer.LocalWriter'>
            enable: True
            max_log_size: 10
            stats_to_track: [EventName.ITER_TRAIN_TIME
            EventName.TRAIN_RAYS_PER_SEC
            EventName.CURR_TEST_PSNR
            EventName.VIS_RAYS_PER_SEC
            EventName.TEST_RAYS_PER_SEC
            EventName.ETA]
        max_buffer_size: 20
        profiler: basic
        relative_log_dir: .
        steps_per_log: 10
    machine: MachineConfig:
        device_type: cuda
        dist_url:

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/5_spins_with_noise/plane-nerf/2023-11-27_101812/nerfstudio_models/step-000009999.ckpt